In [0]:
#import
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
#import lightgbm as lgb
from scipy import stats
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
import math
from scipy.stats import stats
from numpy.random import choice

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#train=pd.read_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/v4_fraud_data_filter_nulls_fill_mean_mode.csv', compression='gzip')
train=pd.read_csv('gdrive/My Drive/Colab Notebooks/Fratrud/Data/fraud_data_remove_nulls_fill_random_distribution.csv', compression='gzip')


In [0]:
train

,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,...,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,_Weekdays,_Hours,_Days,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,device_name,had_id,_Month
0,0,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,552.0,13.0,290.0,222.0,13.0,13.0,0.0,T,T,T,M2,...,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000,-51.0,105328.0,26.0,-89.0,95.065027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,NaN,NaN,NaN,NaN,NaN,1,12
1,0,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,154.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,112.0,13.0,0.0,46.0,0.0,320.0,0.0,NaN,NaN,NaN,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-56.0,353319.0,26.0,-90.0,99.962037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,google,com,NaN,NaN,NaN,1,12
2,0,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,61.0,13.0,0.0,11.0,0.0,315.0,315.0,T,T,T,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-96.0,443506.0,-27.0,-89.0,100.024600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,microsoft,com,NaN,NaN,NaN,1,12
3,0,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,287.0,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,84.0,118.0,111.0,NaN,NaN,NaN,M0,...,0.000000,0.000000,0.000000,50.0,1404.0,790.0,0.000000,0.000000,0.000000,-8.0,843852.0,-7.0,-83.0,96.725812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,2,yahoo,com,NaN,NaN,NaN,1,12
4,0,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,13.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,128.0,11.0,463.0,214.0,185.0,245.0,268.0,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,70787.0,29.0,-89.0,100.000000,NotFound,NaN,New,NotFound,166.0,542.0,144.0,New,NotFound,Samsung,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,5,0,2,google,com,NaN,NaN,Samsung,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0,49.00,W,6550,NaN,150.0,visa,226.0,debit,272.0,87.0,48.0,NaN,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0,29.0,29.0,30.0,264.0,95.0,56.0,56.0,56.0,T,T,T,M0,...,47.950001,47.950001,47.950001,0.0,0.0,0.0,0.000000,0.000000,0.000000,-8.0,552827.0,26.0,-96.0,100.004713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,NaN,NaN,NaN,NaN,NaN,1,6
590536,0,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,16.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,68.0,13.0,0.0,163.0,0.0,0.0,0.0,T,F,F,M0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-56.0,445868.0,26.0,-89.0,100.021605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,1,google,com,NaN,NaN,NaN,1,6
590537,0,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,36.0,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,139.0,56.0,0.0,2.0,0.0,0.0,0.0,T,F,F,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-96.0,11137.0,26.0,-96.0,99.930493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [0]:
train.isnull().sum().sum()

12160821

In [0]:
df_numeric=train._get_numeric_data()
df_describe=df_numeric.describe()
df_describe = df_describe.T

In [0]:
dtype_df=df_numeric.dtypes
df_describe['dtypes']=dtype_df
Cardinality=df_numeric.apply(pd.Series.nunique)
df_describe['Cardinality']=Cardinality
df_skew=df_numeric.skew(axis = 0, skipna = True)
df_describe['Skewtest']=df_skew

In [0]:
df_describe.round(2)

,count,mean,std,min,25%,50%,75%,max,dtypes,Cardinality,Skewtest
isFraud,590540.0,0.03,0.18,0.00,0.00,0.00,0.00,1.00,int64,2,5.06
TransactionDT,590540.0,7372311.31,4617223.65,86400.00,3027057.75,7306527.50,11246620.00,15811131.00,int64,573349,0.13
TransactionAmt,590540.0,135.03,239.16,0.25,43.32,68.77,125.00,31937.39,float64,20902,14.37
card1,590540.0,9898.73,4901.17,1000.00,6019.00,9678.00,14184.00,18396.00,int64,13553,-0.04
card2,590540.0,362.56,156.60,100.00,215.00,361.00,512.00,600.00,float64,501,-0.20
...,...,...,...,...,...,...,...,...,...,...,...
id_11,590540.0,99.75,0.55,90.00,99.75,99.75,99.75,100.00,float64,366,-9.88
id_13,590540.0,48.05,5.47,10.00,48.05,48.05,48.05,64.00,float64,55,-2.70
id_17,590540.0,189.45,14.76,100.00,189.45,189.45,189.45,229.00,float64,105,0.28
id_19,590540.0,353.13,68.53,100.00,353.13,353.13,353.13,671.00,float64,523,0.41


In [0]:
# return all columns with cardinality more than threshold and pass the skew test
def get_cols_for_skew_test(df,threshold):
  cols_for_skew_test=df_describe[df_describe['Cardinality']>threshold].index
  print('There are ' + str(len(cols_for_skew_test)) + ' columns with cardinality higher than '+ str(threshold))
  
  is_skew_cols=set()
  for col in cols_for_skew_test:
    skewness, pvalue = stats.skewtest(df_numeric[col], axis=0, nan_policy = 'omit')
    if pvalue<0.01:
      is_skew_cols.add(col)
  
  print('Out of them, there are ' + str(len(is_skew_cols)) + ' passed the skew test')  
  print(list(is_skew_cols))
  return set(is_skew_cols)

In [0]:
# TODO: 10 is selected just for testin, but we can discuss this further
skew_cols = get_cols_for_skew_test(df_describe,10)
# we don't wish to change the timestamp of transaction, so need to remove column 'TransactionDT'
skew_cols.remove('TransactionDT')
# Please note we suppose always to remove target column (isFraud) as well, but since it's cardinality is 2 it is not part of list

There are 248 columns with cardinality higher than 10
Out of them, there are 248 passed the skew test
['C9', 'V210', 'V245', 'V319', 'V251', 'V187', 'V230', 'V106', 'V186', 'V237', 'V212', 'V238', 'V304', 'V202', 'card3', 'V244', 'C1', 'V250', 'V299', 'V6', 'V40', 'V131', 'V135', 'V279', 'V18', 'card1', 'id_19', 'V125', 'V226', 'V293', 'V273', 'V295', 'V99', 'V26', 'V86', 'V197', 'V169', 'V32', 'C7', 'V198', 'V196', 'V182', 'V129', 'V62', 'V214', 'id_11', 'V270', 'V204', 'V247', 'V280', 'V104', 'V199', 'V228', 'V297', 'D4', 'D2', 'V189', 'V205', 'V253', 'V16', 'V209', 'V257', 'V17', 'V276', 'V132', 'V218', 'V179', 'V101', 'V246', 'V58', 'V303', 'V133', 'V208', 'TransactionDT', 'V123', 'V222', 'V308', 'V310', 'V277', 'addr2', 'V59', 'D3', 'V281', 'V271', 'V264', 'V307', 'V283', 'V229', 'C13', 'V130', 'V3', 'V195', 'V266', 'id_17', 'V220', 'V300', 'V298', 'V39', 'V191', 'V193', 'V265', 'V227', 'D1', 'D15', 'V171', 'V87', 'card5', 'V96', 'V60', 'V309', 'V284', 'V217', 'V278', 'V289', 'V25

In [0]:
pos_skew_cols=skew_cols.intersection(set(df_describe[df_describe['Skewtest']>0].index))
print(len(pos_skew_cols))
print(sorted(pos_skew_cols))

238
['C1', 'C10', 'C11', 'C12', 'C13', 'C14', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D10', 'D11', 'D15', 'D2', 'D3', 'D4', 'D5', 'TransactionAmt', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V111', 'V112', 'V113', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V16', 'V167', 'V168', 'V169', 'V17', 'V170', 'V171', 'V172', 'V175', 'V176', 'V177', 'V178', 'V179', 'V18', 'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V195', 'V196', 'V197', 'V198', 'V199', 'V20', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V23', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V24', 'V242', 'V243', 'V244', 'V245', 'V2

In [0]:
neg_skew_cols=skew_cols.intersection(set(df_describe[df_describe['Skewtest']<0].index))
print(len(neg_skew_cols))
print(sorted(neg_skew_cols))

9
['addr2', 'card1', 'card2', 'card5', 'id_01', 'id_06', 'id_11', 'id_13', 'id_20']


In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,86400,68.50,W,13926,362.555488,150.0,discover,142.0,credit,315.0,87.0,19.00000,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,169.563231,13.000000,140.002441,42.335965,13.000000,13.000000,0.000000,T,T,T,M2,F,T,F,F,T,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.0000,0.000000,0.00000,0.00000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.00000,1.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.542594,0.579198,1.108065,1.1624,0.166076,0.177145,0.999269,0.156118,0.168942,1.083891,1.120779,1.022286,1.038515,0.383174,0.397724,0.164746,0.170579,0.182695,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,117.000000,0.000000,0.0,0.000000,0.0,0.0,117.0,0.0,0.000000,0.000000,0.000000,3.929514,5.859501,0.168053,1.436779,1.690908,0.132012,0.055231,0.127794,0.212639,1.376979,3.529245,6.649283,4.869012,0.924123,0.253332,0.856243,0.484692,0.132364,0.173903,1.148799,1.843752,1.014755,1.038314,1.213083,1.058984,1.237361,1.149845,0.945773,0.954227,1.08458,0.949385,0.961951,1.270749,1.119977,1.159106,444.147142,1078.327538,686.956931,18.060417,6.18936,72.284098,8.888193,35.001872,14.391835,385.137037,765.988339,536.302802,38.437547,133.208217,71.107143,1.054619,1.725784,1.367132,0.169472,1.274912,1.359005,0.093153,0.394748,0.191758,0.247612,0.147975,1.351928,1.63871,1.465476,0.765506,1.005597,0.908388,2.092847,0.184106,0.307667,0.2535,0.127956,0.136965,1.000997,1.000238,1.113463,1.178387,1.118562,0.876619,1.183723,1.025355,1.069493,1.042851,0.788588,0.794231,1.032048,1.162164,1.075113,0.806656,0.81495,1.250993,1.34351,0.967832,0.964425,1.107161,1.013279,117.390676,201.657617,153.520534,9.167839,36.525103,18.813407,5.997508,7.712764,9.445133,8.464571,73.825549

In [0]:
#This loop is taking time... please be patient :)
for col in pos_skew_cols:
  #print(col)
  min_val = df_describe['min'][col]
  #print('min: ' + str(min_val))
  train[col]=train[col].apply(lambda x: np.log(x+abs(min_val)+0.001))

In [0]:
train[list(neg_skew_cols)]=train[list(neg_skew_cols)].apply(lambda x: x*x)

In [0]:
train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,id_01,id_02,id_05,id_06,id_11,id_12,id_13,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,86400,4.230506,W,193933476,131446.482241,5.521465,discover,20164.0,credit,6.028281,7569.0,2.944492,gmail.com,gmail.com,0.001000,0.001000,-6.907755,-6.907755,-6.907755,0.001000,-6.907755,-6.907755,0.001000,-6.907755,0.693647,-6.907755,0.001000,0.001000,2.639129,5.133232,2.565026,5.568358,3.745661,2.565026,4.189670,4.418853,T,T,T,M2,F,T,F,F,T,1.000000,1.000000,0.001000,1.000000,1.000000,0.001000,0.001000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.0000,0.000000,-6.907755,-6.907755,-6.907755,1.000000,0.001000,0.000000,0.000000,0.001000,0.001000,1.00000,0.00100,0.000000,0.00000,0.00000,0.000000,0.000000,-6.907755,0.000000,-6.907755,0.542594,0.579198,0.103517,0.151346,-1.789304,-1.725160,0.999269,0.156118,0.168942,0.081479,0.114916,1.022286,1.038515,0.383174,0.397724,0.164746,0.170579,0.182695,1.000000,1.000000,0.001000,0.001000,0.000000,-6.907755,-6.907755,-6.907755,1.000000,0.001000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.0000,0.000000,0.000000,-6.907755,0.000000,0.000000,1.000000,1.000000,0.001000,0.001000,0.000000,-6.907755,-6.907755,0.00000,0.000000,0.000000,0.000000,0.001000,0.001000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.907755,0.001000,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,0.001000,-6.907755,-6.907755,-6.907755,-6.907755,1.0,1.0,1.0,1.0,0.001,0.001,0.001,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.001,0.001,0.001,-6.907755,4.762182,-6.907755,-6.907755,-6.907755,-6.907755,-6.907755,4.762182,-6.907755,-6.907755,-6.907755,-6.907755,1.368770,1.768235,-1.777543,0.3631,0.525857,-2.017314,0.055231,0.127794,-1.543467,0.320618,1.261367,1.894659,1.583096,-0.077828,-1.369115,-0.154034,-0.722181,-2.014673,-1.743521,0.139587,0.612345,0.015632,0.038561,0.193989,0.058254,0.213789,0.140496,0.945773,-0.045807,0.082114,-0.050889,-0.037753,0.240393,0.1142,0.148511,6.096158,6.983167,6.532273,2.893778,1.822993,4.280618,2.184836,3.555430,2.666731,5.953602,6.641168,6.284701,3.649061,4.891921,4.264202,0.054128,0.546261,0.313446,-1.769186,0.243661,0.307488,-2.362830,-0.926977,-1.646320,-1.391863,-1.903978,0.302271,0.49452,0.382862,-0.265912,0.006575,-0.094984,0.739003,-1.686824,-1.175492,-1.368455,-2.048285,-1.980757,1.000997,1.000238,0.108373,0.164995,0.112937,-0.130542,0.169509,0.026013,0.068119,0

In [0]:
train.isnull().sum().sum()

0

In [0]:
train.to_csv('gdrive/My Drive/Colab Notebooks/Fraud/Data/v1_filterNulls_fill_mean_mode_withTransformation.csv',index=False,compression='gzip')